# README
- 문제 유형 : Text Mining
- 문제 복잡도 : 난이도가 기본적으로 높은 유형. 
    - 주어진 시간과 능력 내에서 해결하기 위해 문제 복잡도를 낮추기 위해 고민
- 데이터 외부 데이터
    - 외부 데이터 : 유튜브 댓글(영화 + 키워드 + 리뷰 형태로 검색), 네이버 영화 평점, 리뷰 
    - 내부 데이터 : 국내/해외 영화 제목 리스트
    
- 문제 해결 방법 :
    - 1. 감정에대한 키워드 사전 생성 (배경지식과 서칭을 활용 or 수작업)
        - 자연어 처리 과정에서 발생할 것으로 예상되는 복잡한 로직 처리를 단순화 하기 위함
    - 2. 수집한 후기, 댓글 데이터에서 정의한 감정 키워드를 단순 카운트 후 매트릭스 생성
    - 3. row : 영화 제목, column : 정의한 감정 키워드
    - 4. 사용 알고리즘 : 
        - BOW : 해당 키워드를 많이 포함하고 있는 영화 TOP10 묶음 추천
        - Clustering : 비지도 학습 기반으로 사람이 찾지 못한 새로운 영화 묶음 인사이트 발견을 위한 방법
            - 인사이트 발견 후 장르 제목을 짓는 작업은 지금 수준으로는 사람의 개입이 필요함, 자동화가 힘들 것으로 예상
            
    - 5. 사용 파이썬 모듈 : 
        - 데이터 수집 : request, ulib
        - 데이터 가공 : pandas, numpy, reg, BeautifulSoup
     
- 트러블 슈팅 및 이슈
    - 네이버 크롤링 : 제목 대신 코드 사용해야함
        - 선행작업 : 영화 코드 크롤링
    - 유튜브 크롤링 : 댓글에서 제목을 파싱해야하는데 일부분 수작업이 필요함
        - 흠..

- 프로젝트의 한계점 및 보완 방법
    - 한계점 : 데이터 커버리지 : 놓치고 있는 부분이 많을 것이다 (자연어 처리 관점이던, 데이터 자체 관점이던)
    - 보완 방법 :
        - 1. 시청자가 직접 배포하는 감성 컨텐츠 묶음
            - 자체 데이터 수집으로서 데이터 퀄리티 및 사용할 수 있는 데이터 수 증가
            - 사용자의 직접 참여로 인해 b tv에 머무는 시간 증가 기대
            
        - 2. b tv 진입시 사용자의 기분을 물어봄
            - 우울/무료/보통/화남/불안/행복/들뜸/신남/ 등..
            - 그리고 시청자의 컨텐츠 사용 패턴, 목록 등에 대한 데이터를 수집
            - 특정 기분일 때 많이 시청하는 컨텐츠를 알 수 있음
            - 또는 특정 기분일 때 시청자들이 함께 많이 보는 컨텐츠를 파악할 수 있음

# 데이터 수집/ 적재 (전처리)
- 1.데이터 필터링 <strong>(v)</strong>
    - 등급 : 19금 제거 <strong>(v)</strong>
    - 제목 (), [], 프리미어, 캐치온 등 제거 <strong>(v)</strong>

- 2.네이버 영화 데이터 수집 <strong>(v)</strong>
    - 수집 : 1. 평점, 2. 코드(제목 매핑), 3. 제목, 4. 리뷰, 5. 한줄평 
        - 평점 5점 이하 영화 필터링  --> 최종 5177개 남았음!!
        - 필터링 된 영화에 한해서 리뷰, 한줄평 수집
    
- 3.유튜브 감성 영화 크리에이터 댓글 데이터 수집
    - 크리에이터 리스트 수집을 하는건 어떨지..
    - 감성 영화 리뷰 라고 검색하고 관련 영상의 댓글들을 일단 모조리 수집해야할라나...
    - 그럴려면 우선 검색 키워드 (감성 키워드)를 먼저 정의한 리스트를 만들자
    

## 제목 전처리 및 19금 영화 제외

### 19금 필터링

In [1]:
import pandas as pd

In [1]:
movieTitle = pd.read_excel('SKB_콘텐츠목록_영화.xlsx',sheet_name= '시청등급추가')

In [2]:
movieTitleNot19 = movieTitle[movieTitle['WAT_GR_CD']!=19][['SRIS_ID','SRIS_NM','WAT_GR_CD']]
movieTitleNot19.head()

,SRIS_ID,SRIS_NM,WAT_GR_CD
0,CS02000001,007 제1탄-살인번호,15
1,CS02000002,007 제10탄-나를 사랑한 스파이,12
2,CS02000003,007 제11탄-문레이커,15
3,CS02000004,007 제12탄-유어 아이스 온리,15
4,CS02000005,007 제13탄-옥토퍼시,15


In [3]:
movieTitleNot19.tail()

,SRIS_ID,SRIS_NM,WAT_GR_CD
22008,CS11032315,진정령 생혼,15
22009,CS11032318,알.이.엠[캐치온],15
22010,CS11032324,무협: 더 코드,15
22011,CS11032383,극장판 반지의 비밀일기[별도편성][예고편],0
22012,CS11032429,왕좌의 전쟁: 드래곤 레전드,15


### 제목 전처리
- 프리이머, 캐치온 [], (), 제\d탄-, 

In [4]:
def removeNosieOnTitle(x):
    import re
    try:
        x = re.sub(r'[[]+\w+[]]|[(]+\w+[)]|[[]+프리미어+\s+오픈[]]','',str(x))
        
        return re.sub(r'제+\d+탄+[-]',' ',x)
    except:
        return x

In [5]:
movieTitleNot19['SRIS_NM_CL'] = movieTitleNot19['SRIS_NM'].apply(removeNosieOnTitle)

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Possible nested set at position 1
  after removing the cwd from sys.path.
c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Possible nested set at position 23
  after removing the cwd from sys.path.


In [131]:
movieTitleNot19.to_csv('cleaned Movie Title.csv')
movieTitleNot19.head()

,SRIS_ID,SRIS_NM,WAT_GR_CD,SRIS_NM_CL
0,CS02000001,007 제1탄-살인번호,15,007 살인번호
1,CS02000002,007 제10탄-나를 사랑한 스파이,12,007 나를 사랑한 스파이
2,CS02000003,007 제11탄-문레이커,15,007 문레이커
3,CS02000004,007 제12탄-유어 아이스 온리,15,007 유어 아이스 온리
4,CS02000005,007 제13탄-옥토퍼시,15,007 옥토퍼시


## 네이버 영화 정보 수집
### 코드 평점 제목 수집

In [105]:
#네이버 영화 랭킹에서 2000개의 양질의 영화를 가져올 수 있음
def getNaverMovieInfo(date : int, pages : int) -> dict():
    '''
    input : date, page수
    output : 제목, 코드, 평점
    '''
    from bs4 import BeautifulSoup as bs
    import requests
    
    movieDict = {}
    movieDict['Score'] = []
    movieDict['Title'] = []
    movieDict['Code'] = []
    
    for page in range(1, pages+1):
        resp= requests.get("https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date="+str(date)+'&page='+str(page))
        soup = bs(resp.text, "html.parser")

        for tr in soup.select('tbody > tr')[1:]:
            
            try:
                movieDict['Score'].append(tr.select('td.point')[0].text)
                movieDict['Title'].append(tr.select('div.tit5 > a')[0].text)
                movieDict['Code'].append(tr.select('div.tit5 > a')[0].attrs['href'].split('code=')[1])
            except:
                continue
        
        if len(movieDict['Score']) > len(movieTitle):
            print(len(movieDic))
            break
            
    return pd.DataFrame(movieDict)

In [147]:
#네이버 검색창에서 리뷰가 정말 없는 영화외에 다 가져올 수 있음
def getAllNaverMovieInfo(titleList : list()) -> dict():
    '''
    input : date, page수
    output : 제목, 코드, 평점
    '''
    from bs4 import BeautifulSoup as bs
    import requests
    
    movieDict = {}
    movieDict['Score'] = []
    movieDict['Title'] = []
    movieDict['Code'] = []
    
    for title in titleList:
        resp= requests.get("https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query="+title)
        soup = bs(resp.text, "html.parser")
        
        #영화 평점, 없으면 0으로 인식
        try:
            movieDict['Score'].append(soup.select('dl.r_grade > dd > em')[0].text)
        except:
            movieDict['Score'].append(0)
        
        #영화 제목 
        movieDict['Title'].append(title)
        
        #영화 코드
        try:
            movieDict['Code'].append(soup.select('div.thumb > a')[0].attrs['href'].split('code=')[1])
        except:
            movieDict['Code'].append('nan')
            
        #print(movieDict)

    return movieDict

In [148]:
titleList= movieTitleNot19['SRIS_NM_CL'].tolist()

print('원본 제목 수 : {}'.format(len(titleList)))
print('중복 제거 제목 수 : {}'.format(len(set(titleList))))

원본 제목 수 : 16383
중복 제거 제목 수 : 11298


In [150]:
movieInfo = getAllNaverMovieInfo(set(titleList))
temp = pd.DataFrame(movieInfo)
temp.to_csv('movie code title score.csv')

In [152]:
movieInfoNew = temp.copy()
movieInfoNew.head()

,Score,Title,Code
0,0,51구역의 비밀,107756
1,0,저스트 렛 고,148690
2,8.81,바람의 소리,70691
3,0,생존자들,nan
4,6.55,슈렉 3,43838


### 평점 5점 이하 필터링 
- 1/2로 줄어듬 

In [125]:
def filterMovie(movieInfo : pd.DataFrame())-> pd.DataFrame():
    '''
    네이버 + 왓챠 평균 평점 5점이하 필터링
    input : movieInfo from function getNaverMovieInfo
    output : 제목, 코드, 평점 테이블 filteredMovieInfo : pd.DataFrame() 
    '''
    filteredMovie = list()
    for naver in movieInfo.iterrows():

        if float(naver[1]['Score']) <= 6:# + watcha['Score']/2 <= 6: #기준은 추후 논의를 통해 수정하는 방향
            pass
        else:
            filteredMovie.append(naver[1]['Title'])
            
    DfMovieInfo = movieInfo.copy()
    filteredMovieInfo = DfMovieInfo[DfMovieInfo['Title'].isin(filteredMovie)]
    
    return filteredMovieInfo

In [205]:
#movieInfo = getNaverMovieInfo(date,pages)
filteredMovieInfo = filterMovie(movieInfoNew)
filteredMovieInfo = filteredMovieInfo[filteredMovieInfo['Code']!='nan']

In [206]:
print('평점 5점 이상 영화 갯수 : {} -> {}'.format(len(movieInfoNew), len(filteredMovieInfo)))
filteredMovieInfo

평점 5점 이상 영화 갯수 : 11298 -> 4907


,Score,Title,Code
2,8.81,바람의 소리,70691
4,6.55,슈렉 3,43838
5,8.76,탈출,17098
9,8.57,킹덤 오브 헤븐,38908
11,9.39,잉여들의 히치하이킹,115298
...,...,...,...
11292,7.31,열두명의 웬수들,37915
11293,8.00,퍼즐,161876
11294,8.59,서극의 칼,18046
11296,9.21,우리들,146504


In [207]:
filteredMovieInfo.to_csv('filtered naver movie_review yes.csv')
print('number of filtered movie : {}'.format(len(filteredMovieInfo)))

number of filtered movie : 4907


- 에러난 영화 따로 빼서 작업

In [169]:
codeNoneMovie = filteredMovieInfo[(filteredMovieInfo['Code'] == 'nan')]
codeNoneMovie.to_csv('code none movie.csv')
codeNoneMovie.head()

,Score,Title,Code
14,6.51,퍼햅스 러브,nan
16,8.96,선물,nan
48,9.00,동화,nan
92,8.29,더 파티,nan
102,8.15,드리프트,nan


- 에러난 영화 코드 가져와서 이어 붙이기

In [213]:
errorMovie = pd.read_csv('code error movie complete.csv',index_col=0)
errorMovie = errorMovie[~errorMovie['Code'].isna()]
errorMovie['Code'] = errorMovie['Code'].astype(int)
errorMovie['Code'] = errorMovie['Code'].astype(str)

totalMovieInfo = pd.concat([filteredMovieInfo[filteredMovieInfo['Code']!='nan'],errorMovie])
#totalMovieInfo = totalMovieInfo[totalMovieInfo['Code']!='nan']
#totalMovieInfo.to_csv('not 19_above 5_all good movie.csv')
totalMovieInfo

,Score,Title,Code
2,8.81,바람의 소리,70691
4,6.55,슈렉 3,43838
5,8.76,탈출,17098
9,8.57,킹덤 오브 헤븐,38908
11,9.39,잉여들의 히치하이킹,115298
...,...,...,...
268,8.64,성 프란치스코,168309
269,10,애니,12238
270,7.77,링컨,65722
271,7.67,일본 대부,13069


### * 최종 5177개의 5점이상 영화 필터링

### 5177개의 영화에 한해서 element, 한줄평 최종 크롤링 --> 5176개 수집

In [25]:
import pandas as pd
totalMovieInfo = pd.read_csv('not 19_above 5_all good movie.csv',index_col=0)
totalMovieInfo.head()

,Score,Title,Code
2,8.81,바람의 소리,70691
4,6.55,슈렉 3,43838
5,8.76,탈출,17098
9,8.57,킹덤 오브 헤븐,38908
11,9.39,잉여들의 히치하이킹,115298


In [14]:
#iframe부분 데이터 받아올 수 있음
def getNaverMovieCommentElementTextIframe(codeList : list())-> pd.DataFrame():
    '''
    input : 영화 코드 리스트
    output : 코드, 한줄평 
    '''
    from bs4 import BeautifulSoup as bs
    import time 
    from selenium import webdriver
    import requests

    #load driver
    driver = webdriver.Chrome(r'C:\Users\owner\hyesung\2020상반기 SK브로드밴드 인턴\CRM기획팀\chromedriver_win32\chromedriver')
    
    #데이터 담을 그릇
    dataDict = dict()
    dataDict['Code'] = []
    dataDict['Element'] = []
    dataDict['Comment'] = []
    
    for code in codeList:
        
        try:
            url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(code)
            driver.get(url)   # 대상 사이트 진입  
            
            ##감상 요소 크롤링
            soupOrigin = bs(driver.page_source)
            elementDict = dict()
            
            try:
                dataDict['Code'].append(code) #code
                for li in soupOrigin.select('div.graph_box > div.grp_sty4 > ul.grp_point > li'):
                    #print(li.select('span.grp_score')[0].text.split('%')[0])
                    
                    element, ratio = li.select('strong')[0].text, int(li.select('span.grp_score')[0].text.split('%')[0])
                    elementDict[element] = ratio

                elementList = sorted(elementDict.items(), key = lambda x: x[1], reverse= True) # 네티즌 투표수(투표 비중) 기준으로 정렬

                elementSelected = str() 
                for e in elementList[:2]: #top2 요소만 선별
                    elementSelected += e[0]+' ' #요소 + 요소 문자형으로 저장

                dataDict['Element'].append(elementSelected) #선별된 element  
                
            except:
                dataDict['Element'].append('')
                print('요소 에러',code)
                continue

            try:

                ##iframe 부분 크롤링 : 한줄평
                driver.switch_to_frame("pointAfterListIframe")    # pointAfterListIframe으로 스위칭 
                soupIframe = bs(driver.page_source)

                comments = str()
                for i, li in enumerate(soupIframe.select('div.score_result > ul >li')):

                    #print(li.select_one('div.score_reple > p > span#_filtered_ment_'+str(i)))
                    comments += li.select_one('div.score_reple > p > span#_filtered_ment_'+str(i)).text.strip() + '\n'  #\n을 구분자로 모든 한줄평을 이어 붙임

                dataDict['Comment'].append(comments)
            except:
                dataDict['Comment'].append('')
                print('한줄평 에러',code)
        except:
            print(code)
            import json
            with open("comment element temp data.json", "w") as json_file:
                json.dump(dataDict, json_file)
            continue

    driver.quit()
    return dataDict

In [6]:
codeList = totalMovieInfo['Code'].tolist() #nan, 0 제외

In [15]:
import time
now = time.time()
commentElementInfo = getNaverMovieCommentElementTextIframe(codeList)
print(time.time() - now)

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: use driver.switch_to.frame instead


한줄평 에러 22080
한줄평 에러 18827
한줄평 에러 18750
1753
한줄평 에러 18316
940100
한줄평 에러 18682
한줄평 에러 18827
2242.672502040863


NameError: name 'tmpSave' is not defined

에러 코드 리스트

In [4]:
errorCodeList = [22080, 18827, 18750, 18316, 18682, 18827]

In [28]:
#codeElmtCmt = pd.DataFrame(commentElementInfo)
#codeElmtCmt.to_csv('comment element temp out data.csv')
codeElmtCmt = pd.read_csv('comment element temp out data.csv',index_col =0)
codeElmtCmt = codeElmtCmt.dropna()
codeElmtCmt['Code'] = codeElmtCmt['Code'].astype(str)

In [22]:
codeElmtCmt[codeElmtCmt['Element'].str.contains('영상')]

,Code,Element,Comment
26,112041,영상미 연출,예고편 만든 사람을 감독으로 추천\n알바입니다 입금 받았으니 십점줍니다 근데 영화보...
28,64080,스토리 영상미,이 영화 보고 딱 한마디.. 역시 사랑은 표현하지 않으면 그리움일뿐...\n영화 중...
30,71901,영상미 OST,배경이나분위기가너무예쁜영화\n우연히 보게 되었는데 동화같고 수채화같은 예쁜 내용이었...
40,163681,영상미 연기,이 영화에서 히로세 스즈를 처음 보았음. 이쁘긴 진짜 이쁨.\n전형적인 학원 멜로....
44,36818,연출 영상미,재밋는데\n6.5점... 결말이 이상해서 6.5점임 ㅇㅇ\n그럭저럭....\n단순 ...
...,...,...,...
5129,97826,영상미 OST,연예인더빙이아니라 정식성우더빙인것이 매우좋고 스토리도 재미있었습니다. 다만 약간아쉬...
5148,85858,연출 영상미,더빙 말고 자막으로 보세요 정말 잘 만들어진 다큐입니다.\n연애인더빙을왜하는거야?\...
5155,15863,영상미 연출,"성룡은 그 자체로 불패신화이고,믿을수 없는 에너지 덩어리이며,극한의완성도 그 자체다..."
5163,17954,연출 영상미,미화라는 것들은 뮤지컬 제대로 안 본 사람들이네. 번듯하게 내세운 정책의 실상이 어...


In [30]:
pd.merge(totalMovieInfo,codeElmtCmt[codeElmtCmt['Element'].str.contains('영상')], on = 'Code').sort_values('Score', ascending=False)

,Score,Title,Code,Element,Comment
474,10.00,디아스포라의 노래: 아리랑로드,190364,연출 영상미,정말 많은분들이 힘든시절을 이겨내고 입에서 입으로 계속 전해지는 아리랑을 듣고 또 ...
114,10.00,모차르트: 프라하의 서곡,163666,연기 영상미,...모자르트의 연인 주자나 역할의 모르피드 클락이 단번에 눈길을 끌었음...물론 ...
433,10.00,파라노말 액티비티 2019,184965,스토리 영상미,나 1등했어요 해헤해헤\n나 2등했어요 해헤해헤\n
19,10.00,선생님과 길고양이,144754,영상미 연출,동물과 사람의 교감은 언제봐도 참 경이롭다.마음 따뜻해짐.\n차가웠던 주인공의 마음...
17,10.00,산타 앤 컴퍼니,171472,영상미 연출,어린이 영화치곤 좀 선정적인 장면이 있어요 프랑스 영화라 그런지 애무 알몸 키스가 ...
...,...,...,...,...,...
519,6.10,투어리스트,51700,연기 영상미,5점까진 아닌듯... 10점까지도 아니지만 나름 볼만했던\n평점 왜케 낮지; 7점 ...
388,6.09,좀비 파이터,186240,연출 영상미,미장센과 드라마에 맡는 음악이 돋보임. 독립영화에서 보기힘든 작품. 하지만 몇몇 배...
513,6.07,아기거북 토토의 바다 대모험,72091,영상미 연출,바다 생물 구경하는 재미가 있을 것 같네요! 25년을 어떻게 다 담았을지 궁금해집니...
169,6.06,폭풍의 언덕,86958,영상미 연출,"작가에 대한 모독이다...캐서린 아역은 말괄량이스러움 대신 후덕함이 돋았고, 성인이..."


한줄평, 요소 에러 난 영화 추가

In [8]:
codeElmtCmt.loc[codeElmtCmt['Code'].isin(errorCodeList),'Element'] = ['연기 연출 스토리', '연기 스토리 연출', '스토리 연출 연기','영상미 연출 연기','영상미 OST 스토리 연출','연기 스토리']
codeElmtCmt.loc[codeElmtCmt['Code'].isin(errorCodeList),'Comment'] = ['웜이 정말 얄미웟음ㅋㅋ근데맷데이먼연기는쵝오\n배우들의 연기가 돋보이는 영화, 도박이란 소재로 적절한 긴장감도 있다.\n타짜의 원조가 여기 있었구나...\n자신의 인생을 찾아 떠날 수 있는 용기를 주는 영화!\n20번을 봐도 질리지 않는 영화, 존말코비치의 러시아인 연기는 최고!\n운명은 네가 선택하는게 아냐. 운명이 널 선택하는거지.\n진짜 재밌었음ㅋㅋ맷 데이먼 팬이기도 하고ㅋㅋ\n맷과 노튼때문에 후하게 준다 ㅋㅋ 연기력은 정말 ㄷㄷ\n도박뿐만 아니라 인생까지 생각하게 되는 영화\n포커에 관한 영화중엔 최고!!',
'알파치노가 서랍장에 돈이 될만한 것들을 넣어두는 그 한장면에 아직까지도 난 이 영화의 여운을 떨쳐내지 못한다.\n알 파치노가 마지막에 보스의 전화를 받고 나가기 전에 머뭇 거리던 모습이 아직도 아련하다.\n나이가 먹고 다시 보니 안보이던것이 보이네요 너무 와닿습니다\n내 인생 최고의 영화\n알파치노 진짜 좋다\n마피아 조직이지만. 이 영화에서 나온 레프티는 그누구보다 측은하고, 인간적인 매력이 넘친다. 보고나면 취한다\n마지막장면 알파치노의 연기는 진짜 미친다\n엄청난 여운... 칼리토를보고 알파치노에 관심이 생겼는데 이젠 푹 빠져버렸다\n내가 영화를 진심으로 좋아하게 된, 알파치노를 좋아하게 된 계기의 영화.\n어떤이는 알파치노의 갱스터영화 중의 다소 떨어진다고는 하지만 조금 더 사실적으로 마피아 세계를 다루고 있어 어쩌면 더 잔인한 전개가 이루어지고 있지 않나 생각한다.긴 러닝타임에 지루하지 않은 편집. 신세계의 미국판이 아닌 도니브래스코의 한국판이 신세계',
'히트 처럼 명작 팝콘 무비인데 평점 낮추는 것들은 뭐야? 에궁, 평점 9점 왔다갔다하는 영화다. 쩝.\n저때 안본 사람들은 말을 말아라,당시엔 스릴러중에 저만한 영화가 없었다.\n97년에 나온 영화였다니 놀랍다. 현재 2010년대의 블록버스터 스릴러물과 비교해도 아쉬운 점이 없는 영화. 배우들 연기는 물론 긴장감과 몰입도까지 최고\n팔짤린애가 불쌍해졌다\n이때만해도 브루스윌리스가 대세였다 본레거시보다 더 낫지 내가 제일 기억나는 장면 거침없이 냉정하게 기관총으로 조준하며 도망가라는 말한마디가 소름끼쳤어\n전문 암살자 당연히 브루스 윌리스가 진리\n잭블랙 다니얼 데 김도 출연하네요 다들 너무 젊어요 ㅋㅋ\n그당시 둘의 배역이 바뀌었음 더 나았겠다하며 봤던 기억ㅡ브루스가 다이하드이후 정의 이미지 강할때여서ㅡ저때가 할리우드의 황금기였던듯.다양한 재미.지금은 맨날 히어로물.지겨움\n난 사실 브루스윌리스가 살기를 바랬다\n기대보다 싱겁게 끝나버린 브루스 윌리스와 리차드 기어의 맞대결',
'레나 올린의 명연기에 섬뜩하죠~^^ ㅎㅎ\n어릴적 강렬하게 남았던 영화 게리와 레나가 아님 이도저도 아닌\n내가 본 최고로 무서운 팜므파탈 레나 올린\n엄청난 팜므파탈이 광기와 전율을 장착하고 인상적으로 전개된다.\n레나 올린 아니었으면 죽었을 영화\n게리올드만 연기는 정말 일품 ..여자분도 소름끼치게 무서웠당\n게리올드만,팜므파탈 레나올린, 줄리엣 루이스 최고 타락한 우리의 자화상\n마지막장면의 게리올드만...왜이렇게 생각나지..레나올린 무서웠다ㅎㅎ\n게리올드만의 영화로 알려져 있으나 이 영화는 완벽히 레나 올린의 영화다\n허술하고 허무하다...',
'어렸을 때 봤던 기억으로 단순한 오락 영화인 줄 알았는데 지금 봐도 정말 수작, 좀 오버하자면 명작이다. CG나 설정, 스토리, 연출도 전반적으로 훌륭.\n테란VS저그 전을 영화로 만듬 ㅋㅋㅋㅋㅋㅋ ㄱ ㅐ명작 진짜 수십번은 봣을듯....토요명화로도 보고 다운받아 보고 여행가서 케이블에서도 보고\n97년도에 누가 이정도 퀄리티 뽑아내갰나 ?\n카르멘이 날 흐믓하게 한다\n저글링개떼 수비하는거 꿀잼, 개인적으로 아바타보다 이게 더재밌음.\n최고의 영화죠 ㅎ 상업 흥행 기억에도 오래남는건 이작품하나임\n진짜 이 감독지금 만들었으면, 더 대박이었을듯,\n꿀잼\n이게 97년작이라는게 믿겨짐?? 어릴때 보고 지금봐도 정말 잘만들었다고 생각함\n왜망했는지 의문이다. 망해서 2편부터는 저예산 홈비디오용으로 촬영되서 시리즈 개판.',
'알파치노가 서랍장에 돈이 될만한 것들을 넣어두는 그 한장면에 아직까지도 난 이 영화의 여운을 떨쳐내지 못한다.\n알 파치노가 마지막에 보스의 전화를 받고 나가기 전에 머뭇 거리던 모습이 아직도 아련하다.\n나이가 먹고 다시 보니 안보이던것이 보이네요 너무 와닿습니다\n내 인생 최고의 영화\n알파치노 진짜 좋다\n마피아 조직이지만. 이 영화에서 나온 레프티는 그누구보다 측은하고, 인간적인 매력이 넘친다. 보고나면 취한다\n마지막장면 알파치노의 연기는 진짜 미친다\n엄청난 여운... 칼리토를보고 알파치노에 관심이 생겼는데 이젠 푹 빠져버렸다\n내가 영화를 진심으로 좋아하게 된, 알파치노를 좋아하게 된 계기의 영화.\n어떤이는 알파치노의 갱스터영화 중의 다소 떨어진다고는 하지만 조금 더 사실적으로 마피아 세계를 다루고 있어 어쩌면 더 잔인한 전개가 이루어지고 있지 않나 생각한다.긴 러닝타임에 지루하지 않은 편집. 신세계의 미국판이 아닌 도니브래스코의 한국판이 신세계']

코드가 잘못된 영화 추가

In [10]:
add = pd.DataFrame({'Code':[73299],'Element':['영상미 OST 연기 연출'],'Comment':['박지윤씨 이뻐요\n괜찮은 영화입니다.\n서울이 이다지도 싱그러운 도시였던가\n별 기대 안 했는데 재미있게 잘 봤어요\n홍상수와 뭔가비슷하면서 좀더 다듬어진듯한 느낌\n박지윤의 전혀다른 아름다움을 보았다...\n박지윤은 시간이갈수록 행보가 멋있다.. 아이돌들이여 본받아라!!!\n서울이란 도시가 이렇게 아름다운줄 몰랐다..\n홍상수감독작품과 비슷하지만 어둡지않고 또한 서울풍경처럼 푸릇한 사랑이 있는것 같다']})
codeElmtCmt = codeElmtCmt.append(add)

In [11]:
codeElmtCmt = codeElmtCmt.reset_index().drop('index',axis=1)
codeElmtCmt

,Code,Element,Comment
0,70691,연기 스토리,이런 명작이 상영관이 없다는 안타까울 따름이다.. 은위같은 졸작에게 스크린 다 내주...
1,43838,스토리 연출,"슈렉시리즈는 다 좋네요...생각지 못한 캐릭터들의 반전도 그렇고, 아이들뿐만 아니라..."
2,17098,연출 스토리,"이연걸의 보디가드, 탈출, 영웅은 꼭 봐야지..!!\n이 당시에 이연걸 영화가 나오..."
3,38908,연출 스토리,이거 감독판 최강이다. 진짜 잊을만 하면 한번씩 보는데 매번 볼때마다 몰입되고 긴장...
4,115298,연출 연기,"정말 실화라고는 믿을 수 없고, 영화라고도 믿을 수 없는 네 청년의 진한 거지냄새 ..."
...,...,...,...
5171,12238,연출 연기,귀도 즐겁고 아이들을 보면서 웃음을 짓게 만든 영화였습니다. 감동적이고 너무너무 재...
5172,65722,연기 연출,이영화는 시답지않은 재미를 논할 수 있는영화가 아니다\n실제의 링컨을 보는 듯한 다...
5173,13069,연출 연기,재일교포들의 설움+시대극을 잘섞은수작!!~근뎅 한국말밖에안나와서 리얼리티-_-\n진...
5174,39411,연출 연기,왜? 난 좋았다 약간 반전이 있었을때는 조금 신선하고 연정훈의 지고지순한 사랑이 느...


csv 저장

In [12]:
#codeElmtCmt.to_csv('comment element temp out data.csv')

### * 여기서 부터 실행하면 됌

In [1]:
import pandas as pd
codeElmtCmt = pd.read_csv('comment element temp out data.csv', index_col=0)
codeElmtCmt

,Code,Element,Comment
0,70691,연기 스토리,이런 명작이 상영관이 없다는 안타까울 따름이다.. 은위같은 졸작에게 스크린 다 내주...
1,43838,스토리 연출,"슈렉시리즈는 다 좋네요...생각지 못한 캐릭터들의 반전도 그렇고, 아이들뿐만 아니라..."
2,17098,연출 스토리,"이연걸의 보디가드, 탈출, 영웅은 꼭 봐야지..!!\n이 당시에 이연걸 영화가 나오..."
3,38908,연출 스토리,이거 감독판 최강이다. 진짜 잊을만 하면 한번씩 보는데 매번 볼때마다 몰입되고 긴장...
4,115298,연출 연기,"정말 실화라고는 믿을 수 없고, 영화라고도 믿을 수 없는 네 청년의 진한 거지냄새 ..."
...,...,...,...
5171,12238,연출 연기,귀도 즐겁고 아이들을 보면서 웃음을 짓게 만든 영화였습니다. 감동적이고 너무너무 재...
5172,65722,연기 연출,이영화는 시답지않은 재미를 논할 수 있는영화가 아니다\n실제의 링컨을 보는 듯한 다...
5173,13069,연출 연기,재일교포들의 설움+시대극을 잘섞은수작!!~근뎅 한국말밖에안나와서 리얼리티-_-\n진...
5174,39411,연출 연기,왜? 난 좋았다 약간 반전이 있었을때는 조금 신선하고 연정훈의 지고지순한 사랑이 느...


### 5176개의 영화에 한해서 리뷰 수집
- comment 수집 과정에서 1개는 drop

In [2]:
def getNaverMovieReviewText(codeList : list())-> dict():
    '''
    수집한 네이버 영화 코드를 이용해서 review text 수집
    input : filteredMovieInfo from function 'filterMovie'
    output : title, review (dictionary) 
    output format : 'Code' : [code1,code2,code3,...], Reveiw : [review1+review2+review3, review1+ review2 + review3...]
    '''
    from bs4 import BeautifulSoup as bs
    from selenium import webdriver as wd 
    import time 
    import requests
    from selenium.webdriver.common.keys import Keys
    import time
    import pyperclip
    import json
    
    ### 네이버 로그인######
    driver = wd.Chrome(r'C:\Users\owner\hyesung\2020상반기 SK브로드밴드 인턴\CRM기획팀\chromedriver_win32\chromedriver')
    driver.get('https://www.naver.com/')
    time.sleep(1)

    # 로그인 버튼을 찾고 클릭합니다
    login_btn = driver.find_element_by_class_name('link_login')
    login_btn.click()
    time.sleep(1)

    # id, pw 입력할 곳을 찾습니다.
    tag_id = driver.find_element_by_name('id')
    tag_pw = driver.find_element_by_name('pw')
    tag_id.clear()
    time.sleep(1)

    # id 입력
    tag_id.click()
    pyperclip.copy('magpie621')
    tag_id.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)

    # pw 입력
    tag_pw.click()
    pyperclip.copy('surgach@1')
    tag_pw.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)

    # 로그인 버튼을 클릭합니다
    login_btn = driver.find_element_by_id('log.login')
    login_btn.click()
    
    
    ### 영화 별로 리뷰 크롤링
    
    
    reviewDict = dict()
    reviewDict['Code'] = []
    reviewDict['Review'] = []
    
    for cut, code in enumerate(codeList):
        try:
            #reviewList = []

            url = url = 'https://movie.naver.com/movie/bi/mi/review.nhn?'+'code='+ str(code)
            driver.get(url)
            #driver.implicitly_wait(2)

            outPageSource= bs(driver.page_source)

            ## 한 영화에 대한 리뷰 리스트 (추천순 TOP10)가 있는 페이지에 접근 
            liList = outPageSource.select('div.review > ul.rvw_list_area > li')
        
            reviews = str()
            for i, _ in enumerate(liList):    

                driver.find_element_by_xpath('//*[@id="reviewTab"]/div/div/ul/li[{}]/p/a'.format(i+1)).click() #리뷰 상세 페이지로 진입
                time.sleep(1)

                inPageSource = bs(driver.page_source)  # html 받아오고 크롤링
                reviews += inPageSource.select_one('div.review > div.top_behavior > strong.h_lst_tx').text.strip()

                pList =  inPageSource.select('div.user_tx_area > p')
                for p in pList:
                    #if 'nbsp' in p.text:
                    reviews += '\n' + p.text.strip()

                driver.back() #다시 뒤로가기
                time.sleep(1) #1초 텀

            reviewDict['Review'].append(reviews)
            reviewDict['Code'].append(code)
            
        except:
            print(code)
        
        if (cut+1)% 50 == 0:
            with open('중간 데이터/moview code review_tail{}.json'.format(str(code)),'w') as jsonFile:
                json.dump(reviewDict, jsonFile)
                
            reviewDict = dict()
            reviewDict['Code'] = []
            reviewDict['Review'] = []
            
    with open('중간 데이터/moview code review last27.json','w') as jsonFile:
        json.dump(reviewDict, jsonFile)
        
    driver.quit()

마지막으로 저장된 코드 다음부터 크롤링하면 됌

In [4]:
codeElmtCmt[codeElmtCmt['Code']==30300] #코드 부분만 바꿔서 실행

,Code,Element,Comment
4799,30300,영상미 스토리,2편이 얼마나 잘만들었는데! 왜 아나킨이 다크사이드를 품게되는지의 원인의 시작. 어...


In [5]:
codeList = codeElmtCmt['Code'].tolist()[4800:] #코드 index +`1 부터 시작

In [6]:
len(codeList)

376

In [8]:
getNaverMovieReviewText(codeList) #run function, 에러난 코드들 메모하기!!

10555
27277
10133
18564


In [ ]:

'''#save data
import json
import time

now = time.time()
with open('moview code review.json','w') as jsonFile:
    json.dump(reviewInfo, jsonFile)
    
print('code run time : ', time.time() - now)'''

In [12]:
a = ''
with open('중간 데이터/moview code review last27.json','r') as jsonFile:
    a = json.load(jsonFile)
dfExmReview = pd.DataFrame(a)
dfExmReview.head()

,Code,Review
0,43681,"설레임의 그 이름, 첫사랑\n\n\n\n\n누구나 살아오면서 첫사랑에 대한 추억을 ..."
1,181103,"""돈 주고 보긴 아까운 영화"" 아이스 시사회 짧은리뷰 스포없음\n\n\n\n안녕하세..."
2,32637,[커먼 웰스] 소리소문없이 대작이 왔다!![커먼웰스]결코 지루하지 않은 영화^^*엽...
3,153374,[여자의 일생]인생은 생각만큼 행복하지도 불행하지도 않군요.\n\n\n\n\n\n\...
4,63535,보고 숙지하여야할 영화\n일본에서 논란의 대상이 된 다큐멘터리 영화 '야스쿠니'가 ...


## 유튜브 감성 영화 크리에이터 댓글 데이터 수집
- 메타 키워드 정리
    - 요소 : 영상, 노래, 연출
    - 상황 : 
    
- 감성 키워드 + 영화 + (리뷰 or 추천) 형식으로 검색
- 댓글에서 추천순 TOP N개 댓글 수집
- ISSUE 
    - 많은 데이터 수집 --> 노이즈가 너무 많음
    - 노이즈 데이터가 많다 --> 일부 수작업 불가피
    - 키워드 수 기반으로 하는게 과연 정확한 것인지?
        - sparse하고 거의 0 또는 1일듯.. 
        - 이를 극복하기 위해 키워드를 '메타'로 가지고 있는 영화를 모두 뽑아서 평점순으로 나열을 해주면 어떨지..
        

In [ ]:
weatherKeyword = ['봄', '여름', '가을', '겨울', '비', '쌀쌀한']
emotionKeyword = ['스트레스 쌓일 때',]
timeKeyword = ['새벽', '출근 길', '퇴근 길']

weatherDecoWord = ['냄새나는','']
emotionKeyword = []
timeDecoWord = ['보기 좋은']

In [ ]:
def getYoutubeEmotionComments():
    '''
    process:
        1. 유튜브 검색창에 '감성 영화 추천' 검색
        2. TOP15개의 컨텐츠 목록에 한해서 컨텐츠 데이터 수집
        3. 수집 데이터 형식 : "제목1 : [댓글1 + 댓글2 + 댓글3,...], 제목2: [댓글1,댓글2,댓글3..]"
    
    '''
    
    return 

# 데이터 처리/통합 (후처리)

## 데이터 통합

<strong>Join commentInfo (코드 한줄 요소) + reviewInfo(코드, 리뷰) + filteredMovieInfo (코드, 제목, 평점) on 네이버 영화 코드</strong>
- <strong> * issue : 리뷰 데이터는 너무 커서 나눠서 돌려야함</strong>

In [2]:
import pandas as pd
movieTitleScore = pd.read_csv('not 19_above 5_all good movie.csv',index_col=0)
movieComment = pd.read_csv('comment element temp out data.csv',index_col=0)
movieComment['Code'] = movieComment['Code'].astype(str)

movieTtleScrElmtCmt = pd.merge(movieTitleScore, movieComment, on = 'Code', how = 'inner').drop_duplicates() #중복 제거
movieTtleScrElmtCmt = movieTtleScrElmtCmt.reset_index().drop('index',axis=1) #인덱스 정렬
movieTtleScrElmtCmt.to_csv('movie title score element comment.csv',encoding='utf-8-sig') #저장
movieTtleScrElmtCmt = pd.read_csv('movie title score element comment.csv',index_col=0) #저장한 것 읽어오기 
movieTtleScrElmtCmt = movieTtleScrElmtCmt.dropna().drop('Element',axis=1)

movieTtleScrElmtCmt 

,Score,Title,Code,Comment
0,8.81,바람의 소리,70691,이런 명작이 상영관이 없다는 안타까울 따름이다.. 은위같은 졸작에게 스크린 다 내주...
1,6.55,슈렉 3,43838,"슈렉시리즈는 다 좋네요...생각지 못한 캐릭터들의 반전도 그렇고, 아이들뿐만 아니라..."
2,8.76,탈출,17098,"이연걸의 보디가드, 탈출, 영웅은 꼭 봐야지..!!\n이 당시에 이연걸 영화가 나오..."
3,8.57,킹덤 오브 헤븐,38908,이거 감독판 최강이다. 진짜 잊을만 하면 한번씩 보는데 매번 볼때마다 몰입되고 긴장...
4,9.39,잉여들의 히치하이킹,115298,"정말 실화라고는 믿을 수 없고, 영화라고도 믿을 수 없는 네 청년의 진한 거지냄새 ..."
...,...,...,...,...
5172,8.64,성 프란치스코,168309,가장 낮은 곳을 향한 열망이 진정한 평화와 기쁨을 가져오는 길임을 알게 해 준 영화...
5173,10.00,애니,12238,귀도 즐겁고 아이들을 보면서 웃음을 짓게 만든 영화였습니다. 감동적이고 너무너무 재...
5174,7.77,링컨,65722,이영화는 시답지않은 재미를 논할 수 있는영화가 아니다\n실제의 링컨을 보는 듯한 다...
5175,7.67,일본 대부,13069,재일교포들의 설움+시대극을 잘섞은수작!!~근뎅 한국말밖에안나와서 리얼리티-_-\n진...


## 데이터 처리

### 메타 키워드 추출

메타 키워드 사전 구축 ex)

In [3]:
metaDict = dict()
metaDict['요소'] ={}
metaDict['감성'] = {}
metaDict['요소']['소리'] = ['OST','노래','사운드']
metaDict['요소']['문화'] = ['일본','할리우드']
metaDict['요소']['연출'] = ['스토리', '각본','연기', '영상미','미장센']
metaDict['감성']['상황'] = ['퇴근 길', '퇴근길','비올 때','비','캠핑','연인','여름','봄','겨울']
metaDict['감성']['감정'] = ['우울','행복']
metaDict

{'요소': {'소리': ['OST', '노래', '사운드'],
  '문화': ['일본', '할리우드'],
  '연출': ['스토리', '각본', '연기', '영상미', '미장센']},
 '감성': {'상황': ['퇴근 길', '퇴근길', '비올 때', '비', '캠핑', '연인', '여름', '봄', '겨울'],
  '감정': ['우울', '행복']}}

사용자 감정 키워드 사전 구축 ex)

In [4]:
twinWordDict = {}
twinWordDict['우울'] =['우울','스트레스','짜증','무기력']
twinWordDict['행복'] = ['행복','신남','즐거움']
#twinWordDict['보통'] = ['보통','그저 그럼','차분','평범']

감정 매칭 영화 키워드 사전 구축 ex)

In [5]:
filterKey1 = dict()
filterKey1['우울'] = ['힐링','위로','격려','치유','위안'] #힐링, 우울감소
filterKey1['행복'] = ['신남','즐거움','행복','빵터짐','웃음','웃김','폭소']
#filterKey['우울2'] = ['울음','슬픔','눈물'] #이울치울
filterKey1

{'우울': ['힐링', '위로', '격려', '치유', '위안'],
 '행복': ['신남', '즐거움', '행복', '빵터짐', '웃음', '웃김', '폭소']}

In [6]:
filterKey2 = dict()
filterKey2['우울2'] = ['울음','슬픔','눈물'] #이울치울
filterKey2['행복'] = ['신남','즐거움','행복','빵터짐','웃음','웃김','폭소']
#filterKey['우울2'] = ['울음','슬픔','눈물'] #이울치울
filterKey2

{'우울2': ['울음', '슬픔', '눈물'], '행복': ['신남', '즐거움', '행복', '빵터짐', '웃음', '웃김', '폭소']}

감성 키워드 추출 함수
- <strong> 시간상, 중점을 두고자 했던 '감정 키워드'만 추출 해보는 것으로 결정 </strong>

In [7]:
def getMetaSentimentKeyWord(x):
    '''
    get word count
    input : 'review' column elements
    output : word count 
    '''
    import re
    
    # 기존에는 단어수 카운트 기반 비중을 보려 했으나 일단은 1개이상 카운트 되면 
    # 해당 키워드를 붙이는 방법으로 진행
    '''frequency = {}    
    for word in matchPattern:
        count = frequency.get(word,0)
        frequency[word] = count+ 1'''
    
    targetMetaList = metaDict['감성']['감정'] # 나중에 바꿔야 하는 부분
    
    returnKey = ''
    for _stmt_ in targetMetaList:
        
        tmpKey = ''
        matchList = filterKey1[_stmt_] #1인지 2인지 명시
        matchPattern = 0
        
        for mKey in matchList:
            matchPattern += len(re.findall(mKey, x))  #동의어 매칭 여부 확인

            if matchPattern == 0: # keyword없으면 return 0
                tmpKey += ''
            else:
                tmpKey += mKey + ' '
                
        if matchPattern != 0 :
            returnKey += 'MetaKey ' + tmpKey + 'Count '+ str(matchPattern) +' '
                
    return returnKey

#### 1.한줄평 

In [65]:
movieTtleScrElmtCmt['Meta'] = movieTtleScrElmtCmt['Comment'].apply(getMetaSentimentKeyWord) # 키워드 추출
movieTtleScrElmtCmt[movieTtleScrElmtCmt['Meta'].str.contains('힐링')] #특정 키워드 영화 선택

,Score,Title,Code,Comment,Meta
30,9.00,리틀 보이,129046,인생은 아름다워가 생각날 정도로 완성도도 높고 재미도 있는 작품-\n남주인공 연기에...,MetaKey 힐링 위로 격려 치유 위안 Count 1
41,7.87,선생님!... 좋아해도 될까요?,163681,이 영화에서 히로세 스즈를 처음 보았음. 이쁘긴 진짜 이쁨.\n전형적인 학원 멜로....,MetaKey 힐링 위로 격려 치유 위안 Count 1
99,9.08,레터스 투 갓,73462,너무나 감동을 주는 영화였습니다!! 8살짜리아이에게서 어른들이 배워가는 가슴따뜻한 ...,MetaKey 힐링 위로 격려 치유 위안 Count 1
122,9.07,델타 보이즈,149479,이런 밑도 끝도 없는 호흡이라니!!! 어마어마한 감독의 탄생을 봤다. 개봉관이 너무...,MetaKey 힐링 위로 격려 치유 위안 Count 2
157,8.50,엘 꼰도르 빠사,94765,남미음악 힐링되네요따뜻한영화에요~\n이런 영화일수록 마이봐야됨 정신건강에 조음......,MetaKey 힐링 위로 격려 치유 위안 Count 3
...,...,...,...,...,...
5096,9.00,대전 블루스,190620,노잼 잠이오는 연기력 비추\n어찌보면 흔한 주제지만 그만큼 많은 사람들이 공감할 이...,MetaKey 힐링 위로 격려 치유 위안 Count 1
5129,7.36,카페,64578,별 뜻없이 가볍게 볼 영화는 아닌듯.. 철학적이고 양자역학같은 이론도 섞여있는데 보...,MetaKey 힐링 위로 격려 치유 위안 Count 2
5139,10.00,여수 밤바다,162149,잔잔하고 소소하지만 때로는 이런 영화가 더 좋을때가 있다\n오랜만에 웃으며 편안하게...,MetaKey 힐링 위로 격려 치유 위안 Count 1 MetaKey 행복 빵터짐 ...
5145,7.84,코블러,118999,설정은 좋으나.. 산으로 가는 전개 그리고 이상한 결말\n초반 30분만 재밌고 볼만...,MetaKey 힐링 위로 격려 치유 위안 Count 1


#### 2.리뷰

In [10]:
import os
reviewFileList = os.listdir('중간 데이터/')
reviewFileList[:5]

['moview code review last27.json',
 'moview code review_tail102278.json',
 'moview code review_tail10253.json',
 'moview code review_tail103304.json',
 'moview code review_tail10405.json']

리뷰 데이터 전체 통합 + Meta Count

In [11]:
reviewSample = pd.DataFrame()
for file in reviewFileList[:100]:
    a = ''
    path = '중간 데이터/' + file
    
    with open(path,'r') as jsonFile:
        a = pd.DataFrame(json.load(jsonFile))
        a['Meta'] = a['Review'].apply(getMetaSentimentKeyWord)
        
    
    reviewSample = pd.concat([reviewSample, a])

<strong>예시 - </strong>감정 : 우울, 매칭 키워드 : [힐링, 위로, 격려, 치유, 위안]

In [139]:
reviewSample[reviewSample['Meta'].str.contains('힐링|위로|격려|치유|위안')]

,Code,Review,Meta
3,153374,[여자의 일생]인생은 생각만큼 행복하지도 불행하지도 않군요.\n\n\n\n\n\n\...,MetaKey 위로 격려 치유 위안 Count 4 MetaKey 눈물 Count 1...
4,63535,보고 숙지하여야할 영화\n일본에서 논란의 대상이 된 다큐멘터리 영화 '야스쿠니'가 ...,MetaKey 위안 Count 2
6,115237,고양이로 시작해 횡설수설 [파스카] 2013 인디포럼 월례비행 10월<파스카> 지독...,MetaKey 위로 격려 치유 위안 Count 3 MetaKey 울음 슬픔 눈물 C...
7,138599,<해에게서 소년에게> 자신을 버린 세상과 어른들에 분노한 어느 한 소년의 이야기 그...,MetaKey 위로 격려 치유 위안 Count 4 MetaKey 행복 빵터짐 웃음 ...
12,183804,[우리지금만나]통일이라는 주제를 재밋게 표현한 독립영화\n\n\n\n\n\n영화 우...,MetaKey 힐링 위로 격려 치유 위안 Count 1
...,...,...,...
40,64192,유행으로 알아본다 : 홍콩영화의 흥망성쇠\n요즘 개봉하는 홍콩영화들은 대부분 극장이...,MetaKey 위로 격려 치유 위안 Count 3 MetaKey 행복 빵터짐 웃음 ...
41,125446,"재키 앤 라이언 (Jackie & Ryan, 2015) 버스킹 시사회 후기(스포X)...",MetaKey 위로 격려 치유 위안 Count 2
44,154536,"사랑하는 법 .. 영화 '아주 긴 변명'아주 긴 변명(永い言い訳, 2016) - 우...",MetaKey 힐링 위로 격려 치유 위안 Count 7 MetaKey 슬픔 눈물 C...
47,77039,[슈퍼에이트] 영화 제대로 이해하기 : 멋진 성장영화!!두 천재 감독의 극비병진 짓...,MetaKey 위로 격려 치유 위안 Count 6 MetaKey 슬픔 눈물 Coun...


sort by count

##### 힐링, 위로, 여운 영화

In [12]:
def getMetaHealingCount(x):
    return int(x.split('MetaKey')[1].split('Count')[1].strip())

In [13]:
sortByCnt = reviewSample[reviewSample['Meta'].str.contains('힐링|위로|격려|치유|위안')]
sortByCnt['Count'] = sortByCnt['Meta'].apply(getMetaHealingCount)

healingMovie = pd.merge(sortByCnt,movieTtleScrElmtCmt[['Title','Code']], on = 'Code', how = 'inner').dropna().drop_duplicates()
healingMovie = healingMovie.rename(columns ={'Review':'Review + Comment'})
healingMovie = healingMovie.sort_values('Count',ascending = False).reset_index().drop('index',axis=1).drop([0])
healingMovie.to_csv('최종 데이터/힐링 영화.csv')

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [167]:
healingMovie.to_csv('healing movie meta table.csv')

##### 우울2 유형 : 이울치울

In [161]:
def getMetaSentimentKeyWord2(x):
    '''
    version 2
    get word count
    input : 'review' column elements
    output : word count 
    '''
    import re
    
    # 기존에는 단어수 카운트 기반 비중을 보려 했으나 일단은 1개이상 카운트 되면 
    # 해당 키워드를 붙이는 방법으로 진행
    '''frequency = {}    
    for word in matchPattern:
        count = frequency.get(word,0)
        frequency[word] = count+ 1'''
    
    targetMetaList = ['우울2','행복'] # 나중에 바꿔야 하는 부분
    
    returnKey = ''
    for _stmt_ in targetMetaList:
        
        tmpKey = ''
        matchList = filterKey2[_stmt_] #1인지 2인지 명시
        matchPattern = 0
        
        for mKey in matchList:
            matchPattern += len(re.findall(mKey, x))  #동의어 매칭 여부 확인

            if matchPattern == 0: # keyword없으면 return 0
                tmpKey += ''
            else:
                tmpKey += mKey + ' '
                
        if matchPattern != 0 :
            returnKey += 'MetaKey ' + tmpKey + 'Count '+ str(matchPattern) +' '
                
    return returnKey

In [162]:
reviewSample2 = pd.DataFrame()
for file in reviewFileList[:100]:
    a = ''
    path = '중간 데이터/' + file
    
    with open(path,'r') as jsonFile:
        a = pd.DataFrame(json.load(jsonFile))
        a['Meta'] = a['Review'].apply(getMetaSentimentKeyWord2)
        
    
    reviewSample2 = pd.concat([reviewSample2, a])

In [165]:
def getMetaSadCount(x):
    return int(x.split('MetaKey')[1].split('Count')[1].strip())

In [166]:
sortByCnt = reviewSample2[reviewSample2['Meta'].str.contains('울음|눈물|슬픔')]
sortByCnt['Count'] = sortByCnt['Meta'].apply(getMetaSadCount)
sadMovie = pd.merge(sortByCnt,movieTtleScrElmtCmt[['Title','Code','Score']], on = 'Code', how = 'inner').dropna().drop_duplicates()
sadMovie = sadMovie.rename(columns ={'Review':'Review + Comment'})
sadMovie.sort_values('Count',ascending = False).reset_index().drop(['index','Code'],axis=1).drop([0]).head(20)

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Review + Comment,Meta,Count,Title,Score
1,너무기대하고봐서인지..\n너무 기대하고 봐서인지 ... 내가 동심을 잃은건지 재미없...,MetaKey 슬픔 눈물 Count 50 MetaKey 즐거움 행복 빵터짐 웃음 웃...,50,인사이드 아웃,9.04
2,시간이 흐르면 바위도 조약돌이 되기에...\n\n\n시간이 흐르면 바위도 조약돌이 ...,MetaKey 울음 슬픔 눈물 Count 29 MetaKey 행복 빵터짐 웃음 웃김...,29,래빗 홀,8.05
3,5.18이 폭동이라고 말하는 사람들아 양심이 있느냐?\n개봉전의 기대치를 보려고 왔...,MetaKey 울음 슬픔 눈물 Count 27 MetaKey 행복 빵터짐 웃음 웃김...,27,화려한 휴가,7.51
4,눈물을 미소로 녹인 영희와 애자의 따뜻한 이별여행!\n\n\n계절이 가을의 문턱으로...,MetaKey 슬픔 눈물 Count 24 MetaKey 웃음 웃김 폭소 Count 18,24,애자,8.75
5,불협화음 [마이보이]\n\n\n불협화음..마이 보이..\n간단하게 줄거리를 보자면....,MetaKey 슬픔 눈물 Count 24,24,마이 보이,8.00
6,하모니 - 잘 만들어진 한편의 뮤직비디오\n\n\n하모니를 보았습니다. '세븐 데이...,MetaKey 울음 슬픔 눈물 Count 23 MetaKey 즐거움 행복 빵터짐 웃...,23,하모니,9.05
7,수준이하 졸작입니다. 돈 낭비하지말고 명절때 tv로 보던지 말던지 ㅋㅋ\n이 영화 ...,MetaKey 울음 슬픔 눈물 Count 23 MetaKey 즐거움 행복 빵터짐 웃...,23,7번방의 선물,8.83
8,"[BIFF] <1993년의 여름, 2017>, 내겐 너무 감당하기 힘든 슬픔.상실을...",MetaKey 울음 슬픔 눈물 Count 21 MetaKey 행복 빵터짐 웃음 웃김...,21,프리다의 그해 여름,9.17
9,로맨스 명가 : 워킹 타이틀의 작품들\n\n어느 한 품목에서 대표 브랜드가 된다는 ...,MetaKey 눈물 Count 21 MetaKey 행복 빵터짐 웃음 웃김 폭소 Co...,21,나의 특별한 사랑이야기,8.29
10,30년간 말을 탄 말전문가의 영화평 그리고 또다른 각설탕스토리..\n전...40대\...,MetaKey 울음 슬픔 눈물 Count 20 MetaKey 즐거움 행복 빵터짐 웃...,20,각설탕,8.84


In [168]:
sadMovie.to_csv('sad movie meta table.csv')

##### 행복해지는 영화

In [ ]:
def getMetaHappyCount(x):
    if len(x.split('MetaKey')) >=3:
        return int(x.split('MetaKey')[2].split('Count')[1].strip())
    else:
        return int(x.split('MetaKey')[1].split('Count')[1].strip())

In [131]:
sortByCnt = reviewSample[reviewSample['Meta'].str.contains('신남|즐거움|행복|빵터짐|웃음|웃김|폭소')]
sortByCnt['Count'] = sortByCnt['Meta'].apply(getMetaHappyCount)

happyMovie = pd.merge(sortByCnt,movieTtleScrElmtCmt[['Title','Code','Score']], on = 'Code', how = 'inner').dropna().drop_duplicates()
happyMovie = happyMovie.rename(columns ={'Review':'Review + Comment'})
happyMovie.sort_values('Count',ascending = False).reset_index().drop(['index','Code'],axis=1).drop([0])[:20]

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Review + Comment,Meta,Count,Title,Score
1,행복한 아빠 윌스미스 그리고 함께 행복했던 두 시간[포스터카피뽀개기] 행복을 찾아서...,MetaKey 격려 치유 위안 Count 1 MetaKey 즐거움 행복 빵터짐 웃음...,75,행복을 찾아서,8.92
2,"먼길을 돌아...너에게 가다그런 게 있는 세상의 , 행복\n\n\n행복하게 살고 싶...",MetaKey 치유 위안 Count 2 MetaKey 행복 빵터짐 웃음 폭소 Cou...,58,행복,7.89
3,"원더풀 라이프(Wonderful Life, 1998) - 지장보살의 은유\n\n\n...",MetaKey 위로 격려 치유 위안 Count 1 MetaKey 행복 빵터짐 웃음 ...,54,원더풀 라이프,8.26
4,영화 조커 JOKER 리뷰\n\n\n\n\n\n누군가 내게 올해 본 영화 중 하나만...,MetaKey 치유 위안 Count 1 MetaKey 행복 빵터짐 웃음 폭소 Cou...,51,조커,8.97
5,보는것만으로도 행복해지는 영화.\n\n\n보는것만으로도 행복해지는 영화가 있다.\n...,MetaKey 위로 격려 치유 위안 Count 20 MetaKey 즐거움 행복 빵터...,50,줄리 앤 줄리아,8.68
6,보는것만으로도 행복해지는 영화.\n\n\n보는것만으로도 행복해지는 영화가 있다.\n...,MetaKey 위로 격려 치유 위안 Count 20 MetaKey 즐거움 행복 빵터...,50,줄리앤줄리아,8.68
7,정말 소재가 신선한 독일 코믹영화\n\n\n\n독일 코미디 영화인 100일동안 10...,MetaKey 힐링 위로 격려 치유 위안 Count 3 MetaKey 행복 빵터짐 ...,49,100일동안 100가지로 100퍼센트 행복찾기,8.63
8,정말 소재가 신선한 독일 코믹영화\n\n\n\n독일 코미디 영화인 100일동안 10...,MetaKey 힐링 위로 격려 치유 위안 Count 3 MetaKey 행복 빵터짐 ...,49,100일 동안 100가지로 100퍼센트 행복찾기,8.63
9,"개똥밭에 굴러도, 살아요\n<스포일러 좀 있음>\n\n이 영화, <내 생애 가장 아...",MetaKey 위안 Count 1 MetaKey 행복 빵터짐 웃음 폭소 Count 48,48,내 생애 가장 아름다운 일주일,8.67
10,"""그래도 간직하고 싶은 영화"" 곰돌이 푸 다시 만나 행복해 짧은리뷰 스포없음\n\n...",MetaKey 힐링 위로 격려 치유 위안 Count 9 MetaKey 행복 빵터짐 ...,46,곰돌이 푸 다시 만나 행복해,8.94


In [125]:
happyMovie[happyMovie['Title'].str.contains('극한')]

,Code,Review + Comment,Meta,Count,Title
1154,167651,극한직업 볼려면 극한 인내력 필요\n너무한거 아니야.. 쌍팔년도 조폭마누라 3편보다...,MetaKey 행복 빵터짐 웃음 폭소 Count 13,13,극한직업


In [169]:
happyMovie.to_csv('happy movie meta table.csv')

In [2]:
import pandas as pd
len(pd.read_csv('최종 데이터/happy movie meta table.csv'))

2747

##### 여름 감성 영화

In [14]:
def getMetaSummerKeyWord(x):
    '''
    get word count
    input : 'review' column elements
    output : word count 
    '''
    import re
    
    # 기존에는 단어수 카운트 기반 비중을 보려 했으나 일단은 1개이상 카운트 되면 
    # 해당 키워드를 붙이는 방법으로 진행
    '''frequency = {}    
    for word in matchPattern:
        count = frequency.get(word,0)
        frequency[word] = count+ 1'''
    
    
    returnKey = ''

    tmpKey = ''
    matchList = ['여름','더울 때','무더위','여름 감성','여름감성','여름 냄새','여름냄새','여름향기','여름 향기','한여름'] #1인지 2인지 명시
    matchPattern = 0

    for mKey in matchList:
        matchPattern += len(re.findall(mKey, x))  #동의어 매칭 여부 확인

        if matchPattern == 0: # keyword없으면 return 0
            tmpKey += ''
        else:
            tmpKey += mKey + ' '

    if matchPattern != 0 :
        returnKey += 'MetaKey ' + tmpKey + 'Count '+ str(matchPattern) +' '
                
    return returnKey

In [15]:
def getMetaSummerCount(x):
    return int(x.split('MetaKey')[1].split('Count')[1].strip())

In [17]:
import pandas as pd
import os
reviewFileList = os.listdir('중간 데이터/')
summerSample = pd.DataFrame()

for file in reviewFileList[:100]:
    a = ''
    path = '중간 데이터/' + file
    
    with open(path,'r') as jsonFile:
        a = pd.DataFrame(json.load(jsonFile))
        a['Meta'] = a['Review'].apply(getMetaSummerKeyWord)
        
    summerSample = pd.concat([summerSample, a])

In [19]:
sortByCnt = summerSample[summerSample['Meta'].str.contains('여름|더울 때|무더위|여름 감성|여름감성|여름 냄새|여름냄새|여름향기|여름 향기|한여름')]
sortByCnt['Count'] = sortByCnt['Meta'].apply(getMetaSummerCount)

summerMovie = pd.merge(sortByCnt,movieTtleScrElmtCmt[['Title','Code','Score']], on = 'Code', how = 'inner').dropna().drop_duplicates()
summerMovie = summerMovie.rename(columns ={'Review':'Review + Comment'})
summerMovie = summerMovie.sort_values('Count',ascending = False).reset_index().drop(['index','Code'],axis=1).drop([0])
summerMovie.to_csv('최종 데이터/키워드_여름,여름냄새,여름향기 영화.csv')
summerMovie[:20]

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Review + Comment,Meta,Count,Title,Score
1,[꼬마 니콜라의 여름방학] - 니콜라의 귀환 환영해!\n\n\n영화 <꼬마 니콜라의...,MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,38,꼬마 니콜라의 여름방학,8.45
2,"사랑은 서핑이다! 떠난 파도를 잊어야 새 파도를 탄다! 임원희와 신소율, 전석호, ...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,33,늦여름,7.94
3,"""건강하고 맛있는 영화"" 리틀 포레스트: 여름과 가을 후기\n\n\n\n\n\n\n...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,32,리틀 포레스트: 여름과 가을,8.64
4,"지난 여름, 갑자기 : 순간의 감성이 부른 마법(이송희일 감독의 연작 퀴어물 Vol...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,28,지난 여름 갑자기,8.36
5,"[두 번째 여름, 두 번 다시 만날 수 없는 너]여주가 너무 귀엽고 노래도 잘하네요...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,27,"두 번째 여름, 두 번 다시 만날 수 없는 너",10.00
6,"무스탕: 랄리의 여름 Mustang, 2015\n\n무스탕: 랄리의 여름 Musta...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,26,무스탕: 랄리의 여름,8.41
7,조니 뎁을 만나보자\n나이트메어(1984)\n조니 뎁(Johnny Depp)이라는 ...,MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,24,캐리비안의 해적-망자의함,8.91
8,조니 뎁을 만나보자\n나이트메어(1984)\n조니 뎁(Johnny Depp)이라는 ...,MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,24,캐리비안의 해적: 망자의 함,8.91
9,"[BIFF] <1993년의 여름, 2017>, 내겐 너무 감당하기 힘든 슬픔.상실을...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,22,프리다의 그해 여름,9.17
10,"영상으로 전하는 바다 판타지 ""해적의제왕""이 되다!!\n\n영화의 시작은 2편 망자...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,21,캐리비안의 해적: 세상의 끝에서,8.21


In [41]:
print(len(summerMovie))

654


In [43]:
summerMovie.sort_values('Count',ascending = False).reset_index().drop(['index','Code'],axis=1).drop([0])[:50]

,Review + Comment,Meta,Count,Title,Score
1,[꼬마 니콜라의 여름방학] - 니콜라의 귀환 환영해!\n\n\n영화 <꼬마 니콜라의...,MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,38,꼬마 니콜라의 여름방학,8.45
2,"사랑은 서핑이다! 떠난 파도를 잊어야 새 파도를 탄다! 임원희와 신소율, 전석호, ...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,33,늦여름,7.94
3,"""건강하고 맛있는 영화"" 리틀 포레스트: 여름과 가을 후기\n\n\n\n\n\n\n...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,32,리틀 포레스트: 여름과 가을,8.64
4,"지난 여름, 갑자기 : 순간의 감성이 부른 마법(이송희일 감독의 연작 퀴어물 Vol...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,28,지난 여름 갑자기,8.36
5,"[두 번째 여름, 두 번 다시 만날 수 없는 너]여주가 너무 귀엽고 노래도 잘하네요...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,27,"두 번째 여름, 두 번 다시 만날 수 없는 너",10.00
6,"무스탕: 랄리의 여름 Mustang, 2015\n\n무스탕: 랄리의 여름 Musta...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,26,무스탕: 랄리의 여름,8.41
7,조니 뎁을 만나보자\n나이트메어(1984)\n조니 뎁(Johnny Depp)이라는 ...,MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,24,캐리비안의 해적-망자의함,8.91
8,조니 뎁을 만나보자\n나이트메어(1984)\n조니 뎁(Johnny Depp)이라는 ...,MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,24,캐리비안의 해적: 망자의 함,8.91
9,"[BIFF] <1993년의 여름, 2017>, 내겐 너무 감당하기 힘든 슬픔.상실을...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,22,프리다의 그해 여름,9.17
10,"영상으로 전하는 바다 판타지 ""해적의제왕""이 되다!!\n\n영화의 시작은 2편 망자...",MetaKey 여름 더울 때 무더위 여름 감성 여름감성 여름 냄새 여름냄새 여름향기...,21,캐리비안의 해적: 세상의 끝에서,8.21


##### 비 올때 감성 영화

In [7]:
def getMetaSituationKeyWord(x):
    '''
    get word count
    input : 'review' column elements
    output : word count 
    '''
    import re
    
    # 기존에는 단어수 카운트 기반 비중을 보려 했으나 일단은 1개이상 카운트 되면 
    # 해당 키워드를 붙이는 방법으로 진행
    '''frequency = {}    
    for word in matchPattern:
        count = frequency.get(word,0)
        frequency[word] = count+ 1'''
    
    
    returnKey = ''

    tmpKey = ''
    matchList = ['비올', '비올 때', '비오는 날', '비내리는 날', '비 내리는날', '비 내리는 날'] #1인지 2인지 명시
    matchPattern = 0

    for mKey in matchList:
        matchPattern += len(re.findall(mKey, x))  #동의어 매칭 여부 확인

        if matchPattern == 0: # keyword없으면 return 0
            tmpKey += ''
        else:
            tmpKey += mKey + ' '

    if matchPattern != 0 :
        returnKey += 'MetaKey ' + tmpKey + 'Count '+ str(matchPattern) +' '
                
    return returnKey

In [16]:
def getMetaRainCount(x):
    return int(x.split('MetaKey')[1].split('Count')[1].strip())

In [9]:
import pandas as pd
import os
reviewFileList = os.listdir('중간 데이터/')
rainSample = pd.DataFrame()

for file in reviewFileList[:100]:
    a = ''
    path = '중간 데이터/' + file
    
    with open(path,'r') as jsonFile:
        a = pd.DataFrame(json.load(jsonFile))
        a['Meta'] = a['Review'].apply(getMetaSituationKeyWord)
        
    
    rainSample = pd.concat([rainSample, a])

In [17]:
sortByCnt = rainSample[rainSample['Meta'].str.contains('비올|비올 때|비오는 날|비내리는 날|비 내리는날|비 내리는 날')]
sortByCnt['Count'] = sortByCnt['Meta'].apply(getMetaRainCount)

rainMovie = pd.merge(sortByCnt,movieTtleScrElmtCmt[['Title','Code','Score']], on = 'Code', how = 'inner').dropna().drop_duplicates()
rainMovie = rainMovie.rename(columns ={'Review':'Review + Comment'})
rainMovie.sort_values('Count',ascending = False).reset_index().drop(['index','Code'],axis=1).drop([0])[:20]

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Review + Comment,Meta,Count,Title,Score
1,행복한 라짜로 (이탈리아에서 더 빛을 발하는 그것의 현대적 우화 )-평점 8점\n칸...,MetaKey 비올 비올 때 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Co...,11,행복한 라짜로,8.74
2,[신작소개] 그녀가 남자라고?\n\n\n도대체 누가 누굴 어떻게 사랑하는거야?\n\...,MetaKey 비올 비올 때 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Co...,8,쉬즈 더 맨,8.88
3,세상의 모든 아침알랭 꼬르노 ▒ 세상의 모든 아침 (Tous Les Matins D...,MetaKey 비올 비올 때 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Co...,8,세상의 모든 아침,8.50
4,조별과제의 폐해는 어떻게 극복되는가.[마지막 4중주] 대한민국에도 이런 영화가 하나...,MetaKey 비올 비올 때 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Co...,4,마지막 4중주,8.80
5,"관객의 뒤통수를 날린 통쾌한 한 방, 김래원과 김해숙의 희생부활자\n\n\n희생부활...",MetaKey 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Count 3,3,희생부활자,6.62
6,"진실, 결론이 없긴 왜없나.ㅡㅡ;; (영화를 보고 보시길~)\n\n\n\n대사와 장...",MetaKey 비올 비올 때 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Co...,2,다우트,8.59
7,"가족간의 삶과 갈등을 다룬 ""걸어도 걸어도""\n\n\n\n\n\n\n\n\n\n고레...",MetaKey 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Count 2,2,걸어도 걸어도,8.83
8,당신을 안다는 것은 불가능한 일일지도 몰라. 전도연 하정우 <멋진 하루>애절한 눈빛...,MetaKey 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Count 2,2,멋진 하루,8.20
9,당신을 안다는 것은 불가능한 일일지도 몰라. 전도연 하정우 <멋진 하루>애절한 눈빛...,MetaKey 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Count 2,2,멋진하루,8.20
10,환타지아(FANTASIA)-1940\n어릴적 국내에서 [인어공주]로부터 인기몰이를 ...,MetaKey 비올 비올 때 비오는 날 비내리는 날 비 내리는날 비 내리는 날 Co...,2,환타지아,9.30


In [18]:
print(len(rainMovie))

54
